<a href="https://colab.research.google.com/github/rajeevnair676/NLP_projects/blob/main/IMDB_review_with_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import nltk
import re
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import accuracy_score,classification_report,roc_auc_score

import transformers
from transformers import DistilBertTokenizer,TFDistilBertModel

from keras.layers import Input,Dense
from keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
# import tensorflow as tf
# print("Tensorflow version " + tf.__version__)

# try:
#   tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
#   print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
# except ValueError:
#   raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

# tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
# #Get a handle to the attached TPU. On GCP it will be the CloudTPU itself
# resolver = tf.distribute.cluster_resolver.TPUClusterResolver()#(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])

# #Connect to the TPU handle and initialise it
# tf.config.experimental_connect_to_cluster(resolver)
# tf.tpu.experimental.initialize_tpu_system(resolver)

In [ ]:
# strategy = tf.distribute.experimental.TPUStrategy(resolver)

In [ ]:
PATH = '/content/drive/MyDrive/NLP/NLP_projects/Classification/IMDB_review_classification/IMDB Dataset.csv'

In [ ]:
def load_data(path):
    df = pd.read_csv(path)
    X_train, X_test, y_train, y_test = train_test_split(df['review'], df['sentiment'], stratify=df['sentiment'],
                                                    test_size=0.33,
                                                    shuffle=True, 
                                                    random_state=42)
    train = pd.concat([X_train,y_train],axis=1)
    test = pd.concat([X_test,y_test],axis=1)
    return train,test

In [ ]:
train,test=load_data(PATH)

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'<br />',' ',text)
    text = re.sub(r'[^A-Za-z0-9]',' ',text)
    text = ' '.join(word for word in text.split())
    return text

In [ ]:
train['cleaned_review']=train.review.apply(lambda s:clean_text(s))
test['cleaned_review']=test.review.apply(lambda s:clean_text(s))

In [ ]:
lb=LabelBinarizer()
train['sentiment']=lb.fit_transform(train['sentiment'])
test['sentiment']=lb.transform(test['sentiment'])

In [ ]:
max_len = train['cleaned_review'].apply(lambda s: len(str(s).split())).max()

In [ ]:
max_len

2505

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(train['cleaned_review'], train['sentiment'],
                                                  stratify=train['sentiment'],
                                                  test_size=0.2,
                                                  random_state=333)

## Tokenizing with DistilBerttokenizer

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

Xtrain = tokenizer(text=list(X_train),
                    truncation=True,
                    add_special_tokens=True,
                    padding=True,
                    return_tensors='tf',
                    return_token_type_ids = False,
                    return_attention_mask=True,
                    verbose=True
                    )

Xval = tokenizer(text=list(X_val),
                    truncation=True,
                    add_special_tokens=True,
                    padding=True,
                    return_tensors='tf',
                    return_token_type_ids = False,
                    return_attention_mask=True,
                    verbose=True
                    )

In [ ]:
Xtrain

{'input_ids': <tf.Tensor: shape=(26800, 512), dtype=int32, numpy=
array([[  101,  1996, 12610, ...,     0,     0,     0],
       [  101,  1999,  1037, ...,     0,     0,     0],
       [  101,  1037,  3689, ...,  2001,  8836,   102],
       ...,
       [  101,  1045,  2572, ...,     0,     0,     0],
       [  101,  2066,  2116, ...,  2336,  1055,   102],
       [  101,  1045,  2320, ...,     0,     0,     0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(26800, 512), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>}

In [ ]:
input_ids = Xtrain['input_ids']
attention_mask = Xtrain['attention_mask']

## Model building with DistilBert

In [ ]:
bert = TFDistilBertModel.from_pretrained('distilbert-base-uncased')

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_layer_norm', 'activation_13', 'vocab_transform', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [ ]:
input_ids = Input(shape=(512,),
                  dtype='int32',
                  name='input_ids')
input_mask = Input(shape=(512,),
                   dtype='int32',
                   name='attention_mask')

embeddings=bert(input_ids,attention_mask=input_mask)[0]

cls_token=embeddings[:,0,:]

outs=Dense(1, activation='sigmoid')(cls_token)

model=Model(inputs=[input_ids,input_mask],
            outputs=outs)

model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 512)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 512)]        0           []                               
                                                                                                  
 tf_distil_bert_model (TFDistil  multiple            66362880    ['input_ids[0][0]',              
 BertModel)                                                       'attention_mask[0][0]']         
                                                                                                  
 tf.__operators__.getitem_2 (Sl  (None, 768)         0           ['tf_distil_bert_model[2][0

In [ ]:
optimizer=Adam(learning_rate=5e-5,
               decay=1e-6)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [ ]:
history = model.fit(
            x={'input_ids':Xtrain['input_ids'],'attention_mask':Xtrain['attention_mask']},
            y=y_train,
            validation_data=({'input_ids':Xval['input_ids'],'attention_mask':Xval['attention_mask']}),
            epochs=3,
            batch_size=8
            )

Epoch 1/3
3350/3350 [==============================] - 1568s 467ms/step - loss: 0.2704 - accuracy: 0.8901 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/3
3350/3350 [==============================] - 1563s 466ms/step - loss: 0.1397 - accuracy: 0.9487 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/3
3350/3350 [==============================] - 1561s 466ms/step - loss: 0.0731 - accuracy: 0.9740 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00


In [ ]:
model.save("/content/drive/MyDrive/NLP/NLP_projects/Classification/IMDB_review_classification/bert_model")